In [5]:
import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [7]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [8]:
# printing the stopwords
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Data Pre-Processing

In [9]:
news_data = pd.read_csv('/content/train.csv')

In [10]:
news_data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1




*   0 ==> Real News
*   1 ==> Fake News



In [11]:
news_data.shape

(20800, 5)

In [12]:
# checking for missing values
news_data.isnull().sum()

,0
id,0
title,558
author,1957
text,39
label,0


In [13]:
news_data.tail()

,id,title,author,text,label
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1
20799,20799,What Keeps the F-35 Alive,David Swanson,"David Swanson is an author, activist, journa...",1


In [14]:
# replacing the missing values with null string
news_data = news_data.fillna('')

In [15]:
news_data.isnull().sum()

,0
id,0
title,0
author,0
text,0
label,0


In [16]:
# merging the author name and news title together
news_data['content'] = news_data['author'] + " " + news_data['title']

In [19]:
news_data.loc[:4, 'content']

,content
0,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,Consortiumnews.com Why the Truth Might Get You...
3,Jessica Purkiss 15 Civilians Killed In Single ...
4,Howard Portnoy Iranian woman jailed for fictio...


In [20]:
# seperating features and targets
X = news_data.drop(columns='label', axis=1)
Y = news_data['label']

In [21]:
Y

,label
0,1
1,0
2,1
3,1
4,1
...,...
20795,0
20796,0
20797,0
20798,1


In [22]:
X.head()

,id,title,author,text,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",Consortiumnews.com Why the Truth Might Get You...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,Howard Portnoy Iranian woman jailed for fictio...


# Stemming
Stemming is the process of reducing a word to its Keyword


*   enjoy ==> [ enjoyable, enjoying ]

Root word is **enjoy**


In [23]:
port_stem = PorterStemmer()

In [40]:
def stemming(content):
  # Convert content to string type to ensure compatibility with re.sub
  content = str(content)
  stemmed_content = re.sub('[^a-zA-Z]', ' ', content)
  lower_content = stemmed_content.lower()
  stemmed_content = lower_content.split()
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)
  return stemmed_content

In [41]:
news_data['content'] = news_data['content'].apply(stemming)

In [42]:
news_data.head()

,id,title,author,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,darrel lucu hou dem aid even see comey letter ...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,daniel j flynn flynn hillari clinton big woman...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,consortiumnew com truth might get fire
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,jessica purkiss civilian kill singl us airstri...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,howard portnoy iranian woman jail fiction unpu...


In [44]:
X = news_data['content'].values
Y = news_data['label'].values

In [32]:
print(X)

[list(['darrel', 'lucu', 'hous', 'dem', 'aid', 'even', 'see', 'comey', 'letter', 'jason', 'chaffetz', 'tweet'])
 list(['daniel', 'j', 'flynn', 'flynn', 'hillari', 'clinton', 'big', 'woman', 'campu', 'breitbart'])
 list(['consortiumnew', 'com', 'truth', 'might', 'get', 'fire']) ...
 list(['michael', 'j', 'de', 'la', 'merc', 'rachel', 'abram', 'maci', 'said', 'receiv', 'takeov', 'approach', 'hudson', 'bay', 'new', 'york', 'time'])
 list(['alex', 'ansari', 'nato', 'russia', 'hold', 'parallel', 'exercis', 'balkan'])
 list(['david', 'swanson', 'keep', 'f', 'aliv'])]


In [33]:
print(Y)

[1 0 1 ... 0 1 1]


In [34]:
Y.shape

(20800,)

In [35]:
X.shape

(20800,)

In [45]:
# converting the textual data to feature vectors
vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)

In [46]:
print(X)

  (0, 263)	0.2701012497770876
  (0, 2464)	0.36765196867972083
  (0, 2933)	0.24684501285337127
  (0, 3567)	0.3598939188262558
  (0, 3759)	0.27053324808454915
  (0, 4917)	0.23331696690935097
  (0, 6930)	0.2187416908935914
  (0, 7612)	0.24785219520671598
  (0, 8546)	0.2921251408704368
  (0, 8822)	0.36359638063260746
  (0, 13348)	0.2565896679337956
  (0, 15553)	0.2848506356272864
  (1, 1481)	0.2957471154505952
  (1, 1877)	0.15614790568229528
  (1, 2206)	0.36915639258038363
  (1, 2790)	0.19208753385709676
  (1, 3535)	0.2653147533915268
  (1, 5440)	0.7186013955384664
  (1, 6744)	0.19152496072048605
  (1, 16656)	0.3025156488372128
  (2, 2917)	0.3179886800654691
  (2, 3072)	0.46097489583229645
  (2, 5326)	0.3866530551182615
  (2, 5903)	0.3474613386728292
  (2, 9532)	0.49351492943649944
  :	:
  (20797, 3610)	0.211655450844435
  (20797, 6969)	0.21809398920480086
  (20797, 8282)	0.22333184464489023
  (20797, 8901)	0.3617803783368178
  (20797, 9431)	0.2939494781564304
  (20797, 9500)	0.17463635692